# IMPORTS

In [0]:
import numpy as np
import pandas as pd
from torch.autograd import Variable
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.datasets as datasets
from torchvision.datasets import MNIST
from torchvision.utils import save_image
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC

# VAE

In [0]:
class vae(nn.Module):
    def __init__(self):
        super(vae, self).__init__()
        
        self.fc1 = nn.Linear(784, 300)
        # self.fc2 = nn.Linear(500, 300)
        self.fcm = nn.Linear(300, 20)
        self.fcs = nn.Linear(300, 20)
        self.fc3 = nn.Linear(20, 300)
        # self.fc4 = nn.Linear(300, 500)
        self.fc5 = nn.Linear(300, 784)

    def encoder(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        mean = torch.relu(self.fcm(x))
        log_std = torch.relu(self.fcs(x))
        return mean, log_std
    
    def reparam(self, mean, log_std):
        std = log_std.mul(0.5).exp_()
        eps = torch.cuda.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        dist_z = eps.mul(std).add_(mean)
        # dist_z = torch.add(dist_z, mean)
        return dist_z

    def decoder(self, z):
        z = torch.relu(self.fc3(z))
        # z = torch.relu(self.fc4(z))
        z = torch.sigmoid(self.fc5(z))
        return z

    def get_latent_dis(self, x):    # returns the latent features
        mean, log_std = self.encoder(x)
        z = self.reparam(mean, log_std)
        return z
    
    def forward(self, x):
        mean, log_std = self.encoder(x)
        z = self.reparam(mean, log_std)
        x1 = self.decoder(z)
        return x1, mean, log_std

model = vae().cuda()
print(model)

In [0]:
num_epochs = 200
batch_size = 64
optimizer = optim.Adam(model.parameters(), lr=1e-3)

img_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5,), (1.0,))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, shuffle=True)

In [0]:
def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_func(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(img)}/{len(dataloader.dataset)}]\tLoss: {loss.item() / len(img)}")

    print(f"====> Epoch: {epoch} Average loss: {train_loss / len(dataloader.dataset)}")
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, str(f"{epoch}.png"))

In [0]:
# torch.save(model.state_dict(), './vae.pth')

model = vae()
model.load_state_dict(torch.load('./vae.pth'))
model.cuda()
model.eval()

# Get New Image from Random Noise

In [0]:
r = torch.randn((64, 28,28))
r = r.view(r.size(0), -1)
r = Variable(r)
img,_,_= model(r.cuda())
save = to_img(img.cpu().data)
save_image(save, 'r.png')

# PLOT


In [0]:
l = []
s = []
for data, target in (dataloader):
    img = data
    img = img.view(img.size(0), -1)
    img = Variable(img)
    l.append(target.numpy()[0])
    if torch.cuda.is_available():
        img = img.cuda()
    z = model.get_latent_dis(img)
    s.append(z.cpu().data.numpy()[0])    

In [0]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 10)
RS = 20150101
X_embedded = TSNE(random_state=RS).fit_transform(s)
sp = sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=l, legend='full', palette=palette)
fig = sp.get_figure()
fig.savefig("./output.png")

# SVM

In [0]:
clf = SVC(gamma='auto')
s = np.array(s)
l = np.array(l)
clf.fit(s, l)

In [0]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5,), (1.0,))
])
valset = datasets.MNIST('./testdata', download=True, train=False, transform=img_transform)
valloader = DataLoader(valset, shuffle=True)

In [0]:
l_test = []
s_test= []

for data, target in (valloader):
    img = data
    img = img.view(img.size(0), -1)
    img = Variable(img)
    l_test.append(target.numpy()[0])
    if torch.cuda.is_available():
        img = img.cuda()
    z = model.get_latent_dis(img)
    s_test.append(z.cpu().data.numpy()[0])   

s_test = np.array(s_test)
l_test = np.array(l_test)

In [0]:
accuracy = clf.score(s_test, l_test)
y_pred = clf.predict(s_test)
precision = precision_score(l_test, y_pred, average=None)
recall = recall_score(l_test, y_pred,  average=None)
f1 = f1_score(l_test, y_pred,  average=None)
cm = confusion_matrix(l_test, y_pred)

print("Accuracy", accuracy)
print("Precision", precision)
print("Recall", recall)
print("f_score", f1)
print("confusion matrix", cm)

In [0]:
# Plot confusion matrix and normalized confusion matrix

class_names = [0,1,2,3,4,5,6,7,8,9]
titles_options = [("Confusion matrix", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(clf, s_test, l_test,
                                 display_labels=class_names,
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)
    print(title)
    print(disp.confusion_matrix)
    plt.savefig(str(f"{title}.png"))
plt.show()